# Practicum OO - AOP
Dit bestand bevat de practicumopgaven voor de module OO&AOP van het vak Advanced Technical Programming (TCIT-VKATP-17). 

Een Jupyter Notebook bestaat uit cells met code of toelichting. Sommige code-cells hoeven enkel uitgevoerd te worden om functies beschikbaar te maken, anderen moet je zelf in vullen (opdrachten staan duidelijk met kopjes gemarkeerd, je moet dan zelf de delen met TODO invullen). Je kunt de code per cel uitvoeren door de cel te selecteren en "run cell" te kiezen. De cel direct hieronder laadt de nodige modules in. Als je code niet naar verwachting werkt kan het zijn dat je deze cell moet runnen, of een van de andere cels vóór de cel met error. Let erop dat als je de Jupyter-server afsluit en later verder gaat moet je deze cells opnieuw runnen.

In [7]:
from unittest import *    
import doctest
import io
import inspect
from contextlib import redirect_stdout

__Opdrachten die zijn gemarkeerd met _[PORTFOLIO]_ dienen te worden opgenomen in je portfolio voor de eindopgave__.

Laat deze opdrachten door je docent aftekenen aan het einde van de les (of aan het begin van de volgende les).

***
## College 2: Reflectie en metaprogrammeren
Deze opdrachten gaan over het college over reflectie en metaprogrammeren en dienen aan het einde van dat college te worden gemaakt.

Maak ook de opdrachten in de reader!


#### Opdracht 2.1 — OO in Python [Portfolio]
Een Queue is gebaseerd op het FIFO-principe: First In First Out

<img src="queue.png" width="500px">

`enqueue` voegt een element achteraan toe.  
`dequeue` haalt een element vooraan weg.

De volgende klasse implementeert Queue:

In [8]:
class MyQueue(list):
    def __init__(self, a=[]):
        list.__init__(self, a)
    
    def dequeue(self):
        return self.pop(0) # geen fout-afhandeling
    
    def enqueue(self, x):
        self.append(x)

Python heeft een aantal ingebouwde queue-klassen: Queue, LifeQueue, PriorityQueue, en deque.

`deque` staat voor double-ended queue: aan beide kanten van de queue kan worden toegevoegd en verwijderd.

Breidt de klasse `MyQueue` uit tot de klasse `MyDeQue`.

De klasse heeft de volgende methoden:
* `appendright(x)`: voeg `x` aan de rechterkant toe
* `appendleft(x)`: voeg `x` aan de linkerkant toe
* `popright()`: verwijder en retourneer het element dat helemaal rechts staat
* `popleft()`: verwijder en retourneer het element dat helemaal links staat
* `reverse()`: keer de elementen in de queue om
* `rotateright(n)`: verschuif de elementen `n` posities naar rechts.
  Als `n < 0`: verschuif de elementen (`-n`)  posities naar links.
* `rotateleft(n)`: verschuif de elementen `n` posities naar links.
  Als `n < 0`: verschuif de elementen (`-n`) posities naar rechts.
  
Maak zoveel mogelijk gebruik van de andere methoden en methoden uit de super-klassen. Zorg voor een goede fout-afhandeling.

Denk er ook aan dat je voldoende tests schrijft om de functionaliteit zoals hierboven gegeven kan garanderen!

In [10]:
class MyDeQue(MyQueue):
    def __init__(self, a=[]):
        super().__init__(a)
    
    def appendright(self, x):
        self.enqueue(x)
    
    def appendleft(self, x):
        self.insert(0, x)
    
    def popright(self):
        if len(self) == 0:
            raise IndexError("pop from an empty MyDeQue")
        return self.pop()
    
    def popleft(self):
        if len(self) == 0:
            raise IndexError("pop from an empty MyDeQue")
        return self.dequeue()
    
    def reverse(self):
        self[:] = self[::-1]
    
    def rotateright(self, n):
        if n >= 0:
            n %= len(self)
            self[:] = self[-n:] + self[:-n]
        else:
            self.rotateleft(-n)
    
    def rotateleft(self, n):
        if n >= 0:
            n %= len(self)
            self[:] = self[n:] + self[:n]
        else:
            self.rotateright(-n)
            
            

def test_MyDeQue():
    # test appendright
    q = MyDeQue([1, 2, 3])
    q.appendright(4)
    assert list(q) == [1, 2, 3, 4]
    
    # test appendleft
    q = MyDeQue([1, 2, 3])
    q.appendleft(0)
    assert list(q) == [0, 1, 2, 3]
    
    # test popright
    q = MyDeQue([1, 2, 3])
    assert q.popright() == 3
    assert list(q) == [1, 2]
    q = MyDeQue([])
    try:
        q.popright()
    except IndexError:
        pass
    else:
        assert False, "popright from empty MyDeQue should raise IndexError"
    
    # test popleft
    q = MyDeQue([1, 2, 3])
    assert q.popleft() == 1
    assert list(q) == [2, 3]
    q = MyDeQue([])
    try:
        q.popleft()
    except IndexError:
        pass
    else:
        assert False, "popleft from empty MyDeQue should raise IndexError"
    
    # test rotateright
    q = MyDeQue([1, 2, 3, 4, 5])
    q.rotateright(2)
    assert list(q) == [4, 5, 1, 2, 3]
    q = MyDeQue([1, 2, 3, 4, 5])
    q.rotateright(-2)
    assert list(q) == [3, 4, 5, 1, 2]
    
    # test rotateleft
    q = MyDeQue([1, 2, 3, 4, 5])
    q.rotateleft(2)
    assert list(q) == [3, 4, 5, 1, 2]
    q = MyDeQue([1, 2, 3, 4, 5])
    q.rotateleft(-2)
    assert list(q) == [4, 5, 1, 2, 3]

    
test_MyDeQue()

#### Opdracht 2.2 — `eval` en veiligheid [Portfolio]

Raadpleeg de site https://www.programiz.com/python-programming/methods/built-in/eval

Een eenvoudige interactieve expressie-evaluator is de volgende:

In [4]:
while True:
    r = input("Type an expression: ")
    if r == "quit": 
        print("exit code")
        break
    print("Result:", eval(r))

Type an expression: quit
exit code


Een dergelijke evaluator is onveilig. Als de gebruiker in een Linux-omgeving de expressie `os.system('rm -rf *')` intikt, kan er veel schade worden aangericht. Je kunt dit voorkomen door bij de eval-functie een dictionary mee te geven.

Schrijf de klasse SimpleSecureEvaluator. De klasse bevat een dictionary-attribuut `d` en een methode `eval`.

De klasse wordt gebruikt bij interactieve sessies, waarbij de gebruiker het volgende kan invoeren:
* een toekenningsopdracht. De opdracht van de vorm `<variabele> = <expressie>` wordt opgeslagen in dictionary-attribuut `d`. Hierbij wordt <variabele> als key-waarde opgeslagen. De expressie wordt eerst geëvalueerd en het resultaat wordt als value-waarde bij de key-waarde opgeslagen. Ook opdrachten van de vorm `i = i +1` zijn toegestaan, maar alleen variabelen van voorafgaande toekenningsopdrachten mogen in de expressie voorkomen.
* een expressie. Alleen variabelen van voorafgaande toekenningsopdrachten mogen in de expressie voorkomen. Bij de evaluatie van de expressie wordt gebruik gemaakt van `d`. Het resultaat wordt aan de gebruiker getoond.
* de opdracht `mydir`. De dictionary `d` wordt dan getoond.

In [ ]:
class SimpleSecureEvaluator(object):
    def __init__(self):
        self.d = {}  # lege dictionary

    def eval(self, expressie):
        if "=" in expressie:
            # toekenningsopdracht
            variabele, expr = expressie.split("=")
            variabele = variabele.strip()
            expr = expr.strip()
            for var in self.d:
                if var in expr:
                    expr = expr.replace(var, str(self.d[var]))
            value = eval(expr)
            self.d[variabele] = value
        elif expressie == "mydir":
            # toon de dictionary
            for key in self.d:
                print(key, "=", self.d[key])
        else:
            # expressie evaluatie
            for var in self.d:
                if var in expressie:
                    expressie = expressie.replace(var, str(self.d[var]))
            print(eval(expressie))

            
sse = SimpleSecureEvaluator()
while True:
    r = input("Type an expression: ")
    if r == "quit": 
        print("exit code")
        break
    sse.eval(r)
    
    
# Output
#============
# Type an expression: i = 1
# Type an expression: b = 2
# Type an expression: i = b + i
# Type an expression: mydir
# i = 3
# b = 2
# Type an expression: quit
# exit code

Type an expression: m a f a = 2
Type an expression: mydir
m a f a = 2


#### Opdracht 2.3.1 — Introspectie van functies
Schrijf een programma dat van een gegeven pyc-file (`function_demo.pyc`) nagaat welke functies de file bevat. Bepaal bij iedere functie de signature en de documentatie. Test de functies.

Ga als volgt te werk:
* Pas de opdracht `dir(<modulename>)` toe. Het resultaat is een lijst van element-namen van de file. De lijst bevat strings.
* Bepaal van ieder element het type. Gebruik hierbij de eval-functie.
* Maak gebruik van het volgende: voor een functie `f` geldt: `type(f).__name__ = 'function'`.
* Gebruikt de functies `signature` en `getdoc` van de module `inspect`.

---------------------------------

In [6]:
# import function_demo_3_5 # Select correct import for your interpreter version
# import function_demo_3_6

# TODO: zie hierboven

import function_demo_3_5
import inspect

# Lijst van element-namen in function_demo.pyc
elementen = dir(function_demo)

# Filter de functies uit de elementenlijst
functies = [eval('function_demo.' + x) for x in elementen if type(eval('function_demo.' + x)).__name__ == 'function']

# Loop door elke functie en print de naam, signature en documentatie
for f in functies:
    print(f.__name__)
    print(inspect.signature(f))
    print(inspect.getdoc(f))
    print('---')

ImportError: bad magic number in 'function_demo_3_5': b'\x16\r\r\n'

#### Opdracht 2.3.2 — Introspectie van klassen
Schrijf een programma dat van de gegeven pyc-file (`class_demo.pyc`) nagaat welke klassen de file bevat.

Ga te werk zoals bij opdracht 2.3.1.

Bedenk daarbij het volgende:
* Voor een klasse `C` geldt: `type(c).__name__ == 'type'`
* Voor een methode `m` geldt: `type(m).__name__ == 'function'`

In [ ]:
# import class_demo_3_5
# import class_demo_3_6

# TODO: zie hierboven

#### Opdracht 2.4 — Decorators [ Portfolio]
Een decorator is een functie die andere functies aanpast om zodanig (aspect-oriented) functionaliteit toe te voegen. Denk bijvoorbeeld aan de optie op een timing toe te  voegen (hoe lang duurt het uitvoeren van een bepaalde functie-aanroep), of security aspecten (alleen als het juiste pincode/password wordt ingevoerd na aanroep zal de functie ook echt worden uitgevoerd).

Het basispatroon van een decorator is het volgende:

In [17]:
from functools import wraps
import time

def myDecorator(f):
    # Definieer een binnenste functie, die je om de 'oude' functie heen 'wrapt'
    @wraps(f) #Gebruik van wraps niet noodzakelijk (zie reader), maar wel netter
    def inner(*args, **kwargs):
        start_time = time.time()  # Record the start time
        result = f(*args, **kwargs)
        end_time = time.time()  # Record the end time

        execution_time = end_time - start_time
        print("Execution time:", execution_time, "seconds")

        return result
    return inner

# Toepassen d.m.v. '@' alleen mogelijk bij definitie van de functie!
@myDecorator
def successor(x):
    time.sleep(1)
    return x+1

print(successor(3))
print('-'*20)

# Maar het kan ook door 'overschrijven' van eerder gedefinieerde functie (= hernoemen van de functie-pointer)
from math import sqrt
print(sqrt(4)) # nog niets verandert

print('-'*20)

sqrt = myDecorator(sqrt)
print(sqrt(4)) # nu wel!

Execution time: 1.012918472290039 seconds
4
--------------------
2.0
--------------------
Execution time: 0.0 seconds
2.0


Herschrijf de decorator hierboven zodat hij naar het scherm schrijft welke functie (naam) uit welke module (naam) er gedraait is. Je kan deze decorator vervolgens toepassen bij de volgende opdracht, als je kan achterhalen hoe je hem toevoegd aan __alle__ functies van __alle__ modules van de simulator.

#### Eindopdracht
Gebruik de beschrijving van de simulator in hoofdstuk 6.1 van de reader en introspectie om:
* Het functioneren van de simulator te achterhalen;
* Tests te kunnen draaien om de correctheid van de simulator te achterhalen (functioneert de simulator vergelijkbaar met de fysieke opstelling?);
* De controller unit van de simulator te vervangen door een eigen geschreven variant.

De Python-byte code van de simulator is toegevoegd in de folder `simulator-3.5` (voor Python 3.5) en `simulator-3.6` (voor Python 3.6). Gebruik `main.py` om de simulator te starten.   
De GUI vereist de installatie van PyGame; gebruik `pip3 install pygame` in een prompt om de pygame-module te installeren (draai als Admin op Windows).

In [ ]:
from simulator_3_6 import main

main()